In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from pennylane.templates import RandomLayers
n_epochs = 30
n_qubits=4
np.random.seed(0)
tf.random.set_seed(0)
dev = qml.device("default.qubit", wires=4)
params = 2 * np.pi * tf.random.uniform([3,50])

In [ ]:
def dataset(dataset_no,train_size,test_size):
    n_train = train_size
    n_test = test_size
    if dataset_no==1:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    elif dataset_no==2:
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    train_images = x_train[:n_train]
    train_labels = y_train[:n_train]
    test_images = x_test[:n_test]
    test_labels = y_test[:n_test]
    train_images = train_images / 255
    test_images = test_images / 255
    train_images = np.array(train_images[..., tf.newaxis],requires_grad=False)
    test_images = np.array(test_images[..., tf.newaxis],requires_grad=False)

    return train_images,test_images,train_labels,test_labels

In [ ]:
dev = qml.device("default.qubit", wires=4)

In [ ]:
@qml.qnode(dev)
def circuit_1(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi * data[j], wires=j)

    for i in range(n_qubits):
        qml.RX(params[1,i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    for i in range(n_qubits):
        qml.RX(params[2,i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]


In [ ]:
@qml.qnode(dev)
def circuit_2(data,params):
    for j in range(n_qubits):
            qml.RX(np.pi * data[j], wires=j)

    for j in range(n_qubits):
            qml.Rot(params[1,3*j],params[1,3*j+1],params[1,3*j+2], wires=j)
    for j in range(0, n_qubits):
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])

                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])


    for j in range(n_qubits):
            qml.Rot(params[2,3*j],params[2,3*j+1],params[2,3*j+2], wires=j)

    for j in range(0, n_qubits):

                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])

                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

In [ ]:

@qml.qnode(dev)
def circuit_3(data,params):
    for i in range(0, n_qubits):
        qml.RY(np.pi * data[i], wires=i)

    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)
    qml.CRX(params[1,n_qubits], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1, 0, -1):
        qml.CRX(params[1,n_qubits+i], wires=[i - 1, i])

    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)
    qml.CRX(params[2,n_qubits], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1, 0, -1):
        qml.CRX(params[2,n_qubits+i], wires=[i - 1, i])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

In [ ]:

@qml.qnode(dev)
def circuit_4(data,params):
    for i in range(0, n_qubits):
        qml.RY(np.pi * data[i], wires=i)


    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)

    for i in range(0,n_qubits - 1,2):
        qml.CRX(params[1,i], wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)
        qml.RZ(params[1,i], wires=i)

    for i in range(1,n_qubits - 1,2):
        qml.CRX(params[1,i], wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)

    for i in range(0,n_qubits - 1,2):
        qml.CRX(params[2,i], wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)
        qml.RZ(params[2,i], wires=i)

    for i in range(1,n_qubits - 1,2):
        qml.CRX(params[2,i], wires=[i, i+1])

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]


In [ ]:

@qml.qnode(dev)
def circuit_5(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi*data[j], wires=j)

    for j in range(0, n_qubits):
        qml.U3(params[1,j], params[1,j + 1], params[1,j + 2], wires=j)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    for j in range(0, int(n_qubits / 2)):
        qml.RY(params[1,j], wires=2 * j)
        qml.RZ(params[1,j], wires=2 * j + 1)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i,i+1])
    for j in range(0, int(n_qubits / 2)):
        qml.RY(params[1,j], wires=2 * j)
    for i in range(0, n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    for j in range(0, n_qubits):
        qml.U3(params[2,j], params[2,j + 1], params[2,j + 2], wires=j)

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]

In [ ]:

@qml.qnode(dev)
def circuit_6(data,params):
    for j in range(n_qubits):
        qml.RY(np.pi*data[j], wires=j)

    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)
    for i in range(n_qubits-1,0,-1):
        qml.CZ(wires=[i,i-1])
    for i in range(0, n_qubits):
        qml.RX(params[1,i], wires=i)


    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)
    for i in range(n_qubits-1,0,-1):
        qml.CZ(wires=[i,i-1])
    for i in range(0, n_qubits):
        qml.RX(params[2,i], wires=i)

    return [qml.expval(qml.PauliZ(w)) for w in range(4)]


In [ ]:
def quanv(image,circuit_no):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))
    if circuit_no==1:
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_1(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]

    elif circuit_no==2:
         for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_2(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]

    elif circuit_no==3:

         for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_3(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]

    elif circuit_no==4:

        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_4(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)
                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]

    elif circuit_no==5:

        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_5(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)

                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]


    elif circuit_no==6:
        for j in range(0, 28, 2):
            for k in range(0, 28, 2):
                q_results = circuit_6(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params)

                for i in range(4):
                    out[j // 2, k // 2, i] = q_results[i]


    return out


In [ ]:
train_images,test_images,train_labels,test_labels=dataset(2,250,60)

In [ ]:

q_train_images = []
print("Quantum pre-processing of train images:")
for idx, img in enumerate(train_images):
    print("{}/{}        ".format(idx + 1, 250), end="\r")
    q_train_images.append(quanv(img,3)) #please select the circuit number
q_train_images = np.asarray(q_train_images)

q_test_images = []
print("\nQuantum pre-processing of test images:")
for idx, img in enumerate(test_images):
    print("{}/{}        ".format(idx + 1, 60), end="\r")
    q_test_images.append(quanv(img,3))
q_test_images = np.asarray(q_test_images)



Quantum pre-processing of train images:


/usr/local/lib/python3.10/dist-packages/pennylane/math/utils.py:223: UserWarning: Contains tensors of types {'autograd', 'tensorflow'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(



Quantum pre-processing of test images:


In [ ]:


model = tf.keras.models.Sequential([
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

opt=tf.keras.optimizers.Adam(learning_rate=0.01)


model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)



In [ ]:
model1 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),input_shape=(28,28,1),activation='relu'),
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        #tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)

In [ ]:
q_history = model.fit(q_train_images,train_labels,validation_data=(q_test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)

Epoch 1/30
16/16 - 1s - loss: 2.5121 - accuracy: 0.1280 - val_loss: 1.9772 - val_accuracy: 0.1333 - 1s/epoch - 71ms/step
Epoch 2/30
16/16 - 0s - loss: 1.8520 - accuracy: 0.2040 - val_loss: 1.6057 - val_accuracy: 0.3833 - 82ms/epoch - 5ms/step
Epoch 3/30
16/16 - 0s - loss: 1.6770 - accuracy: 0.2920 - val_loss: 1.4695 - val_accuracy: 0.4833 - 72ms/epoch - 5ms/step
Epoch 4/30
16/16 - 0s - loss: 1.5454 - accuracy: 0.3760 - val_loss: 1.4534 - val_accuracy: 0.4500 - 82ms/epoch - 5ms/step
Epoch 5/30
16/16 - 0s - loss: 1.4486 - accuracy: 0.4080 - val_loss: 1.2500 - val_accuracy: 0.5833 - 65ms/epoch - 4ms/step
Epoch 6/30
16/16 - 0s - loss: 1.4240 - accuracy: 0.4240 - val_loss: 1.3055 - val_accuracy: 0.5167 - 80ms/epoch - 5ms/step
Epoch 7/30
16/16 - 0s - loss: 1.3085 - accuracy: 0.4320 - val_loss: 1.3172 - val_accuracy: 0.5167 - 74ms/epoch - 5ms/step
Epoch 8/30
16/16 - 0s - loss: 1.3879 - accuracy: 0.4360 - val_loss: 1.1952 - val_accuracy: 0.5000 - 69ms/epoch - 4ms/step
Epoch 9/30
16/16 - 0s - l

In [ ]:
c_history = model1.fit(train_images,train_labels,validation_data=(test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)

Epoch 1/30
16/16 - 1s - loss: 1.8374 - accuracy: 0.3680 - val_loss: 1.2557 - val_accuracy: 0.6000 - 1s/epoch - 83ms/step
Epoch 2/30
16/16 - 0s - loss: 0.8479 - accuracy: 0.7120 - val_loss: 1.2996 - val_accuracy: 0.5833 - 343ms/epoch - 21ms/step
Epoch 3/30
16/16 - 0s - loss: 0.6179 - accuracy: 0.7840 - val_loss: 1.2356 - val_accuracy: 0.6500 - 382ms/epoch - 24ms/step
Epoch 4/30
16/16 - 0s - loss: 0.4698 - accuracy: 0.8320 - val_loss: 1.2674 - val_accuracy: 0.6167 - 393ms/epoch - 25ms/step
Epoch 5/30
16/16 - 0s - loss: 0.4162 - accuracy: 0.8400 - val_loss: 1.5004 - val_accuracy: 0.6833 - 385ms/epoch - 24ms/step
Epoch 6/30
16/16 - 0s - loss: 0.3344 - accuracy: 0.8640 - val_loss: 1.4610 - val_accuracy: 0.6500 - 384ms/epoch - 24ms/step
Epoch 7/30
16/16 - 0s - loss: 0.3085 - accuracy: 0.8840 - val_loss: 1.6354 - val_accuracy: 0.6833 - 372ms/epoch - 23ms/step
Epoch 8/30
16/16 - 0s - loss: 0.2496 - accuracy: 0.8920 - val_loss: 1.7389 - val_accuracy: 0.6833 - 352ms/epoch - 22ms/step
Epoch 9/30
